In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import os
import random

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from scipy.stats import skew, kurtosis

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.decomposition import PCA

# import sys
# sys.path.append('../input/moa-models-and-packages/site-packages')
# from gauss_rank_scaler import GaussRankScaler

SEED = 721991

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)    
    tf.random.set_seed(seed)

In [2]:


# Reading the data
df_train = pd.read_csv('./train_features.csv')
df_test = pd.read_csv('./test_features.csv')
df_train_targets_scored = pd.read_csv('./train_targets_scored.csv')
df_train_targets_nonscored = pd.read_csv('./train_targets_nonscored.csv')
df_test_targets = pd.read_csv('./sample_submission.csv')

# Identifying target features
target_features_scored = list(df_train_targets_scored.columns[1:])
target_features_nonscored = list(df_train_targets_nonscored.columns[1:])

# Identifying columns to retain that have at least 4 non-zero entries
columns_to_retain_nonscored = [col for col in df_train_targets_nonscored.columns if (df_train_targets_nonscored[col] != 0).sum() >= 4]

# Selecting only the retained columns
df_train_targets_nonscored = df_train_targets_nonscored[columns_to_retain_nonscored]

# Updating the list of target features for nonscored targets
target_features_nonscored = [col for col in columns_to_retain_nonscored if col != 'sig_id']  # Assuming 'sig_id' is not a target feature

# Type conversion for optimization
df_train_targets_scored[target_features_scored] = df_train_targets_scored[target_features_scored].astype(np.uint8)
df_train_targets_nonscored[target_features_nonscored] = df_train_targets_nonscored[target_features_nonscored].astype(np.uint8)
df_test_targets[target_features_scored] = df_test_targets[target_features_scored].astype(np.float32)


In [3]:
df_train = df_train.merge(df_train_targets_scored, on='sig_id', how='left')
df_train = df_train.merge(df_train_targets_nonscored, on='sig_id', how='left')
df_test = df_test.merge(df_test_targets, on='sig_id', how='left')

print(f'Training Set Shape = {df_train.shape}')
print(f'Training Set Memory Usage = {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Test Set Shape = {df_test.shape}')
print(f'Test Set Memory Usage = {df_test.memory_usage().sum() / 1024 ** 2:.2f} MB')

Training Set Shape = (23814, 1408)
Training Set Memory Usage = 171.24 MB
Test Set Shape = (3982, 1082)
Test Set Memory Usage = 29.74 MB


In [4]:
df_train.head()
# sig_id is the unique sample id
# Features with g- prefix are gene expression features and there are 772 of them (from g-0 to g-771)
# Features with c- prefix are cell viability features and there are 100 of them (from c-0 to c-99)
# cp_type is a binary categorical feature which indicates the samples are treated with a compound or with a control perturbation (trt_cp or ctl_vehicle)
# cp_time is a categorical feature which indicates the treatment duration (24, 48 or 72 hours)
# cp_dose is a binary categorical feature which indicates the dose is low or high (D1 or D2)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,...,cytokine_production_inhibitor,dehydrogenase_inhibitor,diacylglycerol_kinase_inhibitor,diacylglycerol_o_acyltransferase_inhibitor,differentiation_inducer,dihydroorotate_dehydrogenase_inhibitor,dihydropteroate_synthase_inhibitor,dihydropyrimidine_dehydrogenase_inhibitor,dna_dependent_protein_kinase_inhibitor,dna_methyltransferase_inhibitor,dna_polymerase_inhibitor,dna_synthesis_inhibitor,dopamine_release_enhancer,dot1l_inhibitor,dynamin_inhibitor,dyrk_inhibitor,endothelin_receptor_antagonist,enkephalinase_inhibitor,ephrin_inhibitor,epoxide_hydolase_inhibitor,eukaryotic_translation_initiation_factor_inhibitor,exportin_antagonist,fabi_inhibitor,fatty_acid_receptor_antagonist,fatty_acid_synthase_inhibitor,folate_receptor_ligand,fungal_ergosterol_inhibitor,fungal_lanosterol_demethylase_inhibitor,fxr_agonist,g_protein-coupled_receptor_agonist,g_protein-coupled_receptor_antagonist,g_protein_signaling_inhibitor,gaba_gated_chloride_channel_blocker,gaba_receptor_modulator,gaba_uptake_inhibitor,gap_junction_modulator,gastrin_inhibitor,gat_inhibitor,gli_antagonist,glp_receptor_agonist,glucagon_receptor_antagonist,glucocorticoid_receptor_antagonist,gluconeogenesis_inhibitor,glucose_dependent_insulinotropic_receptor_agonist,glucosidase_inhibitor,glutamate_receptor_modulator,glutathione_reductase_(nadph)_activators,glycine_receptor_antagonist,glycine_transporter_inhibitor,glycogen_phosphorylase_inhibitor,glycosylation_inhibitor,gonadotropin_receptor_antagonist,gtpase_inhibitor,guanylate_cyclase_activator,guanylyl_cyclase_activator,haemostatic_agent,hcn_channel_antagonist,hedgehog_pathway_inhibitor,heme_oxygenase_activators,hexokinase_inhibitor,hgf_receptor_inhibitor,hif_inhibitor,histamine_release_inhibitor,histone_acetyltransferase_inhibitor,histone_demethylase_inhibitor,hiv_integrase_inhibitor,hiv_protease_inhibitor,hydantoin_antiepileptic,hydroxycarboxylic_acid_receptor_agonist,icam1_antagonist,icam1_inhibitor,id1_expression_inhibitor,imidazoline_ligand,immunostimulant,"indoleamine_2,3-dioxygenase_inhibitor",inosine_monophosphate_dehydrogenase_inhibitor,inositol_monophosphatase_inhibitor,interferon_inducer,interleukin_inhibitor,interleukin_receptor_agonist,ion_channel_antagonist,ip1_prostacyclin_receptor_agonist,isocitrate_dehydrogenase_inhibitor,jnk_inhibitor,kainate_receptor_antagonist,katp_activator,keap1_ligand,kinesin_inhibitor,lactamase_inhibitor,lactate_dehydrogenase_inhibitor,lanosterol_demethylase_inhibitor,leucyl-trna_synthetase_inhibitor,leukocyte_elastase_inhibitor,lim_inhibitor,lipase_clearing_factor_inhibitor,lipid_peroxidase_inhibitor,lipoprotein_lip

In [5]:
g_features = [feature for feature in df_train.columns if feature.startswith('g-')]
c_features = [feature for feature in df_train.columns if feature.startswith('c-')]
other_features = [feature for feature in df_train.columns if feature not in g_features and 
                                                             feature not in c_features and 
                                                             feature not in target_features_scored and
                                                             feature not in target_features_nonscored]

print(f'Number of g- Features: {len(g_features)}')
print(f'Number of c- Features: {len(c_features)}')
print(f'Number of Other Features: {len(other_features)} ({other_features})')

Number of g- Features: 772
Number of c- Features: 100
Number of Other Features: 4 (['sig_id', 'cp_type', 'cp_time', 'cp_dose'])


In [6]:
# check for any NA Values
df_train.isna().sum().sum()

0

## Dataset Operations

In [7]:
df_train

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,...,cytokine_production_inhibitor,dehydrogenase_inhibitor,diacylglycerol_kinase_inhibitor,diacylglycerol_o_acyltransferase_inhibitor,differentiation_inducer,dihydroorotate_dehydrogenase_inhibitor,dihydropteroate_synthase_inhibitor,dihydropyrimidine_dehydrogenase_inhibitor,dna_dependent_protein_kinase_inhibitor,dna_methyltransferase_inhibitor,dna_polymerase_inhibitor,dna_synthesis_inhibitor,dopamine_release_enhancer,dot1l_inhibitor,dynamin_inhibitor,dyrk_inhibitor,endothelin_receptor_antagonist,enkephalinase_inhibitor,ephrin_inhibitor,epoxide_hydolase_inhibitor,eukaryotic_translation_initiation_factor_inhibitor,exportin_antagonist,fabi_inhibitor,fatty_acid_receptor_antagonist,fatty_acid_synthase_inhibitor,folate_receptor_ligand,fungal_ergosterol_inhibitor,fungal_lanosterol_demethylase_inhibitor,fxr_agonist,g_protein-coupled_receptor_agonist,g_protein-coupled_receptor_antagonist,g_protein_signaling_inhibitor,gaba_gated_chloride_channel_blocker,gaba_receptor_modulator,gaba_uptake_inhibitor,gap_junction_modulator,gastrin_inhibitor,gat_inhibitor,gli_antagonist,glp_receptor_agonist,glucagon_receptor_antagonist,glucocorticoid_receptor_antagonist,gluconeogenesis_inhibitor,glucose_dependent_insulinotropic_receptor_agonist,glucosidase_inhibitor,glutamate_receptor_modulator,glutathione_reductase_(nadph)_activators,glycine_receptor_antagonist,glycine_transporter_inhibitor,glycogen_phosphorylase_inhibitor,glycosylation_inhibitor,gonadotropin_receptor_antagonist,gtpase_inhibitor,guanylate_cyclase_activator,guanylyl_cyclase_activator,haemostatic_agent,hcn_channel_antagonist,hedgehog_pathway_inhibitor,heme_oxygenase_activators,hexokinase_inhibitor,hgf_receptor_inhibitor,hif_inhibitor,histamine_release_inhibitor,histone_acetyltransferase_inhibitor,histone_demethylase_inhibitor,hiv_integrase_inhibitor,hiv_protease_inhibitor,hydantoin_antiepileptic,hydroxycarboxylic_acid_receptor_agonist,icam1_antagonist,icam1_inhibitor,id1_expression_inhibitor,imidazoline_ligand,immunostimulant,"indoleamine_2,3-dioxygenase_inhibitor",inosine_monophosphate_dehydrogenase_inhibitor,inositol_monophosphatase_inhibitor,interferon_inducer,interleukin_inhibitor,interleukin_receptor_agonist,ion_channel_antagonist,ip1_prostacyclin_receptor_agonist,isocitrate_dehydrogenase_inhibitor,jnk_inhibitor,kainate_receptor_antagonist,katp_activator,keap1_ligand,kinesin_inhibitor,lactamase_inhibitor,lactate_dehydrogenase_inhibitor,lanosterol_demethylase_inhibitor,leucyl-trna_synthetase_inhibitor,leukocyte_elastase_inhibitor,lim_inhibitor,lipase_clearing_factor_inhibitor,lipid_peroxidase_inhibitor,lipoprotein_lip

In [8]:
# Shuffle the dataset
# df_train = shuffle(df_train, random_state=42)

# Drop the 'sig_id' column
df_train = df_train.drop('sig_id', axis=1)

# One-hot encoding for 'cp_type', 'cp_time', 'cp_dose'
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_columns = one_hot_encoder.fit_transform(df_train[['cp_type', 'cp_time', 'cp_dose']])
column_names = one_hot_encoder.get_feature_names_out(['cp_type', 'cp_time', 'cp_dose'])

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_columns, columns=column_names)

c:\Users\chopr\anaconda3\envs\part-time\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
# Drop original columns and concat the new encoded columns
df_train = df_train.drop(['cp_type', 'cp_time', 'cp_dose'], axis=1)
df_train = pd.concat([df_train, encoded_df], axis=1)

In [10]:
# Identify the one-hot encoded features for cp_type, cp_time, and cp_dose
one_hot_features = [feature for feature in df_train.columns if feature.startswith('cp_type_') or 
                                                                feature.startswith('cp_time_') or 
                                                                feature.startswith('cp_dose_')]


# Combine all feature lists to create the X dataset
feature_columns = g_features + c_features + one_hot_features
X = df_train[feature_columns]

# For Y, use the columns that are not in the features list
Y = df_train.drop(feature_columns, axis=1)

print(f'Number of g- Features: {len(g_features)}')
print(f'Number of c- Features: {len(c_features)}')
print(f'Number of One Hot Features: {len(one_hot_features)} ({one_hot_features})')


Number of g- Features: 772
Number of c- Features: 100
Number of One Hot Features: 7 (['cp_type_ctl_vehicle', 'cp_type_trt_cp', 'cp_time_24', 'cp_time_48', 'cp_time_72', 'cp_dose_D1', 'cp_dose_D2'])


In [11]:
# Check Data Consistency
# Features should add up
print((len(target_features_scored) + len(target_features_nonscored))==Y.shape[1])
# entire data should add up
print(df_train.shape[1] == (1484 - 4 + 7)) #(original shape, - 4 ( 3 categorical features + sig_id) + 7 )
# the difference in Y and entire dataset should just be the features 
print(df_train.shape[1] - Y.shape[1] == len(g_features)+len(c_features)+len(one_hot_features))

True
False
True


In [12]:
df_train

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,...,dihydropyrimidine_dehydrogenase_inhibitor,dna_dependent_protein_kinase_inhibitor,dna_methyltransferase_inhibitor,dna_polymerase_inhibitor,dna_synthesis_inhibitor,dopamine_release_enhancer,dot1l_inhibitor,dynamin_inhibitor,dyrk_inhibitor,endothelin_receptor_antagonist,enkephalinase_inhibitor,ephrin_inhibitor,epoxide_hydolase_inhibitor,eukaryotic_translation_initiation_factor_inhibitor,exportin_antagonist,fabi_inhibitor,fatty_acid_receptor_antagonist,fatty_acid_synthase_inhibitor,folate_receptor_ligand,fungal_ergosterol_inhibitor,fungal_lanosterol_demethylase_inhibitor,fxr_agonist,g_protein-coupled_receptor_agonist,g_protein-coupled_receptor_antagonist,g_protein_signaling_inhibitor,gaba_gated_chloride_channel_blocker,gaba_receptor_modulator,gaba_uptake_inhibitor,gap_junction_modulator,gastrin_inhibitor,gat_inhibitor,gli_antagonist,glp_receptor_agonist,glucagon_receptor_antagonist,glucocorticoid_receptor_antagonist,gluconeogenesis_inhibitor,glucose_dependent_insulinotropic_receptor_agonist,glucosidase_inhibitor,glutamate_receptor_modulator,glutathione_reductase_(nadph)_activators,glycine_receptor_antagonist,glycine_transporter_inhibitor,glycogen_phosphorylase_inhibitor,glycosylation_inhibitor,gonadotropin_receptor_antagonist,gtpase_inhibitor,guanylate_cyclase_activator,guanylyl_cyclase_activator,haemostatic_agent,hcn_channel_antagonist,hedgehog_pathway_inhibitor,heme_oxygenase_activators,hexokinase_inhibitor,hgf_receptor_inhibitor,hif_inhibitor,histamine_release_inhibitor,histone_acetyltransferase_inhibitor,histone_demethylase_inhibitor,hiv_integrase_inhibitor,hiv_protease_inhibitor,hydantoin_antiepileptic,hydroxycarboxylic_acid_receptor_agonist,icam1_antagonist,icam1_inhibitor,id1_expression_inhibitor,imidazoline_ligand,immunostimulant,"indoleamine_2,3-dioxygenase_inhibitor",inosine_monophosphate_dehydrogenase_inhibitor,inositol_monophosphatase_inhibitor,interferon_inducer,interleukin_inhibitor,interleukin_receptor_agonist,ion_channel_antagonist,ip1_prostacyclin_receptor_agonist,isocitrate_dehydrogenase_inhibitor,jnk_inhibitor,kainate_receptor_antagonist,katp_activator,keap1_ligand,kinesin_inhibitor,lactamase_inhibitor,lactate_dehydrogenase_inhibitor,lanosterol_demethylase_inhibitor,leucyl-trna_synthetase_inhibitor,leukocyte_elastase_inhibitor,lim_inhibitor,lipase_clearing_factor_inhibitor,lipid_peroxidase_inhibitor,lipoprotein_lipase_activator,lrkk2_inhibitor,lymphocyte_inhibitor,lysophosphatidic_acid_receptor_antagonist,macrophage_inhibitor,macrophage_migration_inhibiting_factor_inhibitor,map_k,map_kinase_inhibitor,matrix_metalloprotease_inhibitor,mcl1_inhibi

## Basline ML model

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Separate the DataFrame into continuous and one-hot encoded features
X_continuous = X[g_features + c_features]
X_one_hot = X[one_hot_features]

# Scale the continuous features
scaler = StandardScaler()
X_continuous_scaled = scaler.fit_transform(X_continuous)

# Combine the scaled continuous features and one-hot encoded features
X_scaled = np.concatenate([X_continuous_scaled, X_one_hot], axis=1)

# Verify that X and X_scaled have the same dimensions
print(X_scaled.shape == X.shape)


True


In [17]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)
# Model
model = MultiOutputClassifier(LogisticRegression())
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
score = f1_score(y_test, y_pred, average='micro') # Choose an appropriate average method
print('F1 Score:', score)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0